<a href="https://colab.research.google.com/github/QianyueWang0212/mgmt467-analytics-portfolio/blob/main/Labs/Lab7_Batch_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 7: Automated Batch Pipeline (Cloud Functions 2nd Gen + Cloud Scheduler)
Date generated: 2025-08-21

**Objective:** Build a serverless, automated data pipeline that fetches API data and loads it to BigQuery on a schedule.  
**Stack:** Cloud Functions (2nd gen), Secret Manager, BigQuery, Cloud Scheduler.

**Deliverables:** (a) This notebook with outputs, (b) scheduler screenshot, (c) BigQuery table screenshot.

In [17]:
# Optional (Colab): install gcloud and libs
# !apt -y -qq install google-cloud-sdk
# !pip -q install google-cloud-bigquery google-cloud-secret-manager requests
PROJECT_ID = "mgmt-46700"
REGION     = "us-central1"
DATASET    = "superstore_data"
TABLE      = "realtime_weather"
CITY       = "Lafayette,IN,US"
print(PROJECT_ID, REGION, DATASET, TABLE, CITY)

mgmt-46700 us-central1 superstore_data realtime_weather Lafayette,IN,US


### Enable required APIs (run once per project)

In [2]:
# !gcloud services enable run.googleapis.com cloudfunctions.googleapis.com bigquery.googleapis.com secretmanager.googleapis.com cloudscheduler.googleapis.com logging.googleapis.com --project $PROJECT_ID

### Create/verify BigQuery dataset & table

In [18]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
# dataset
try:
    client.get_dataset(f"{PROJECT_ID}.{DATASET}")
except Exception:
    client.create_dataset(bigquery.Dataset(f"{PROJECT_ID}.{DATASET}"), exists_ok=True)
# table schema
schema = [
    bigquery.SchemaField("ingest_ts", "TIMESTAMP"),
    bigquery.SchemaField("city", "STRING"),
    bigquery.SchemaField("weather", "STRING"),
    bigquery.SchemaField("temp_c", "FLOAT"),
    bigquery.SchemaField("humidity", "FLOAT"),
    bigquery.SchemaField("wind_mps", "FLOAT"),
    bigquery.SchemaField("raw", "JSON"),
]
table_ref = bigquery.Table(f"{PROJECT_ID}.{DATASET}.{TABLE}", schema=schema)
client.create_table(table_ref, exists_ok=True)
print("Ready:", f"{PROJECT_ID}.{DATASET}.{TABLE}")

Ready: mgmt-46700.superstore_data.realtime_weather


### Secret Manager: create `OWM_API_KEY` secret and add your API key

In [15]:
# from google.cloud import secretmanager
# sm = secretmanager.SecretManagerServiceClient()
# parent = f"projects/{PROJECT_ID}"
# secret_id = "OWM_API_KEY"
# try:
#     sm.get_secret(request={"name": f"{parent}/secrets/{secret_id}"})
# except Exception:
#     sm.create_secret(request={"parent": parent, "secret_id": secret_id, "secret": {"replication": {"automatic": {}}}})
# sm.add_secret_version(request={"parent": f"{parent}/secrets/{secret_id}", "payload": {"data": b"YOUR_OPENWEATHERMAP_API_KEY"}})
# print("Secret ready:", secret_id)

### Cloud Function (Python) — copy this into `main.py` when deploying
> Create `requirements.txt` with:  
`google-cloud-bigquery>=3.25.0`  
`google-cloud-secret-manager>=2.20.2`  
`requests>=2.31.0`

In [22]:
# --- BEGIN: main.py template ---
import os, json, logging, datetime, requests
from google.cloud import bigquery, secretmanager

PROJECT_ID = os.environ.get("PROJECT_ID")
DATASET    = os.environ.get("DATASET")
TABLE      = os.environ.get("TABLE")
CITY       = os.environ.get("CITY", "Lafayette,IN,US")
SECRET_ID  = os.environ.get("OWM_SECRET_ID", "OWM_API_KEY")

_bq = bigquery.Client()
_sm = secretmanager.SecretManagerServiceClient()
_api_key_cache = None

def _api_key():
    global _api_key_cache
    if _api_key_cache: return _api_key_cache
    name = f"projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/latest"
    resp = _sm.access_secret_version(request={"name": name})
    _api_key_cache = resp.payload.data.decode()
    return _api_key_cache

def weather_ingest(request):
    try:
        params = {"q": CITY, "appid": _api_key(), "units": "metric"}
        r = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params, timeout=15)
        r.raise_for_status()
        data = r.json()
        row = {
            "ingest_ts": datetime.datetime.utcnow().isoformat()+"Z",
            "city": CITY,
            "weather": (data.get("weather") or [{}])[0].get("main"),
            "temp_c": (data.get("main") or {}).get("temp"),
            "humidity": (data.get("main") or {}).get("humidity"),
            "wind_mps": (data.get("wind") or {}).get("speed"),
            "raw": data,
        }
        table_id = f"{PROJECT_ID}.{DATASET}.{TABLE}"
        errors = _bq.insert_rows_json(table_id, [row])
        if errors:
            logging.error(errors); return (json.dumps({"ok": False, "errors": errors}), 500, {"Content-Type":"application/json"})
        return (json.dumps({"ok": True, "rows": 1}), 200, {"Content-Type":"application/json"})
    except Exception as e:
        logging.exception("error")
        return (json.dumps({"ok": False, "error": str(e)}), 500, {"Content-Type":"application/json"})
# --- END: main.py template ---

### Deploy (2nd gen) and set env vars

In [23]:
# !gcloud functions deploy weather_ingest --gen2 --region=$REGION --runtime=python312 --source=. #    --entry-point=weather_ingest --trigger-http --no-allow-unauthenticated #    --set-env-vars=PROJECT_ID=$PROJECT_ID,DATASET=$DATASET,TABLE=$TABLE,CITY="$CITY",OWM_SECRET_ID=OWM_API_KEY

### Grant Secret Accessor to the function's service account and schedule hourly with Cloud Scheduler (OIDC)

In [24]:
# FUNCTION_SA=$(gcloud functions describe weather_ingest --gen2 --region=$REGION --format="value(serviceConfig.serviceAccountEmail)")
# gcloud secrets add-iam-policy-binding OWM_API_KEY --member="serviceAccount:${FUNCTION_SA}" --role="roles/secretmanager.secretAccessor"
# URL=$(gcloud functions describe weather_ingest --gen2 --region=$REGION --format="value(serviceConfig.uri)")
# gcloud iam service-accounts create scheduler-invoker --display-name="Scheduler Invoker" --project $PROJECT_ID
# gcloud run services add-iam-policy-binding $(basename $URL) --region=$REGION --member="serviceAccount:scheduler-invoker@$PROJECT_ID.iam.gserviceaccount.com" --role="roles/run.invoker"
# gcloud scheduler jobs create http weather-ingest-hourly --schedule="0 * * * *" --uri="$URL" --http-method=POST --oidc-service-account-email="scheduler-invoker@$PROJECT_ID.iam.gserviceaccount.com" --oidc-token-audience="$URL"

### Validate recent rows in BigQuery

In [28]:
import os
from google.cloud import bigquery

# Ensure PROJECT_ID and DATASET are set (added for robustness in case previous cells were not run)
PROJECT_ID = "mgmt-46700"
DATASET    = "superstore_data"
TABLE      = "realtime_weather"

# Ensure PROJECT_ID is set as an environment variable for BigQuery client to pick up
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

client = bigquery.Client(project=PROJECT_ID)
for row in client.query(f"SELECT ingest_ts, city, weather, temp_c FROM `{PROJECT_ID}.{DATASET}.{TABLE}` ORDER BY ingest_ts DESC LIMIT 5"):
    print(dict(row))

### Challenge (author a Gemini prompt)
Write a prompt to add try/except for `requests.exceptions.RequestException` and structured logging with `google-cloud-logging`. Place it here:

Write an improved version of the weather_ingest Cloud Function by adding try/except around requests.get to catch requests.exceptions.RequestException, adding structured logging using google-cloud-logging (initialize with google.cloud.logging.Client().setup_logging()), using a logger named "weather_ingest" with structured fields (city, status, row_count, api_status_code), and adding logs for function start, API start, API success, API failure, BigQuery insert success, and insert failure. Return JSON with "ok", "message", and "ingested_rows". Output only the updated weather_ingest function and nothing else. The code must run in Cloud Functions 2nd Gen Python 3.12.